In [3]:
import os
import cv2
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [9]:
path= "./chest_xray"
train_file_0= os.listdir(path+ "/train/NORMAL")
train_file_1= os.listdir(path+ "/train/PNEUMONIA")
test_file_0= os.listdir(path+ "/test/NORMAL")
test_file_1= os.listdir(path+ "/test/PNEUMONIA")

In [15]:
X_full= 0
check= False

for file in train_file_0:
    img_in= cv2.imread(path+ "/train/NORMAL/"+ file, cv2.IMREAD_GRAYSCALE)
    img_res= cv2.resize(img_in, dsize= (100, 100), interpolation= cv2.INTER_LINEAR)
    img_round= img_res/255
    img_resh= img_round.reshape((100, 100, 1))
    img= img_resh.reshape((1, )+ img_resh.shape)
    
    if(check== False):
        X_full= img
        check= True
    else:
        X_full= np.vstack((X_full, img))

for file in train_file_1:
    img_in= cv2.imread(path+ "/train/PNEUMONIA/"+ file, cv2.IMREAD_GRAYSCALE)
    img_res= cv2.resize(img_in, dsize= (100, 100), interpolation= cv2.INTER_LINEAR)
    img_round= img_res/255
    img_resh= img_round.reshape((100, 100, 1))
    img= img_resh.reshape((1, )+ img_resh.shape)
    X_full= np.vstack((X_full, img))


In [16]:
y_full= np.array([0]* len(train_file_0))
y_full_1= np.array([1]* len(train_file_1))
y_full= np.append(y_full, y_full_1)


In [17]:
X_test, y_train= shuffle(X_full, y_full, random_state= 42)

In [18]:
X_train, X_val, y_train, y_val= train_test_split(X_full, y_full, test_size= 0.12)

In [19]:
np.save("./dataset/X_train", X_train)
np.save("./dataset/y_train", y_train)
np.save("./dataset/X_val", X_val)
np.save("./dataset/y_val", y_val)

In [20]:
X_test= 0
check= False

for file in test_file_0:
    img_in= cv2.imread(path+ "/test/NORMAL/"+ file, cv2.IMREAD_GRAYSCALE)
    img_res= cv2.resize(img_in, dsize= (100, 100), interpolation= cv2.INTER_LINEAR)
    img_round= img_res/255
    img_resh= img_round.reshape((100, 100, 1))
    img= img_resh.reshape((1, )+ img_resh.shape)
    
    if(check== False):
        X_test= img
        check= True
    else:
        X_test= np.vstack((X_test, img))

for file in test_file_1:
    img_in= cv2.imread(path+ "/test/PNEUMONIA/"+ file, cv2.IMREAD_GRAYSCALE)
    img_res= cv2.resize(img_in, dsize= (100, 100), interpolation= cv2.INTER_LINEAR)
    img_round= img_res/255
    img_resh= img_round.reshape((100, 100, 1))
    img= img_resh.reshape((1, )+ img_resh.shape)
    X_test= np.vstack((X_test, img))


In [21]:
y_test= np.array([0]* len(test_file_0))
y_test_1= np.array([1]* len(test_file_1))
y_test= np.append(y_test, y_test_1)

In [22]:
X_test, y_test= shuffle(X_test, y_test, random_state= 42)

In [23]:
np.save("./dataset/X_test", X_test)
np.save("./dataset/y_test", y_test)

In [24]:
X_train= np.load("./dataset/X_train.npy")
y_train= np.load("./dataset/y_train.npy")
X_val= np.load("./dataset/X_val.npy")
y_val= np.load("./dataset/y_val.npy")
X_test= np.load("./dataset/X_test.npy")
y_test= np.load("./dataset/y_test.npy")

In [25]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)


(4590, 100, 100, 1)
(4590,)
(626, 100, 100, 1)
(626,)
(624, 100, 100, 1)
(624,)


In [43]:
model= keras.models.Sequential()

In [44]:
model.add(keras.layers.Conv2D(64, 10, activation= "relu", padding= "same", input_shape= [100, 100, 1]))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(128, 4, activation= "relu", padding= "same"))


model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(64, activation= "relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(32, activation= "relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation= "softmax"))



In [45]:
model.compile(loss= "sparse_categorical_crossentropy", optimizer= "adam", metrics= ["accuracy"])

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 64)      6464      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 50, 128)       131200    
_________________________________________________________________
flatten (Flatten)            (None, 320000)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                20480064  
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2

In [47]:
weight_for_0 = 1.94
weight_for_1 = 0.67

class_weight = {0: weight_for_0, 1: weight_for_1}

In [48]:
with tf.device('/GPU:0'):
    from keras.callbacks import EarlyStopping
    es=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=40)

    start_time= time.time()
    history=model.fit(X_train, y_train, epochs= 80, validation_data= (X_val, y_val), class_weight=class_weight)
    end_time= time.time()
    print("time: ", end_time- start_time)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/80


2022-12-12 14:34:32.591184: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-12 14:34:32.591371: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


144/144 [==============================] - 43s 297ms/step - loss: 0.6860 - accuracy: 0.6168 - val_loss: 0.3716 - val_accuracy: 0.8706
Epoch 2/80
144/144 [==============================] - 41s 286ms/step - loss: 0.4662 - accuracy: 0.8096 - val_loss: 0.4545 - val_accuracy: 0.7907
Epoch 3/80
144/144 [==============================] - 41s 284ms/step - loss: 0.3469 - accuracy: 0.8636 - val_loss: 0.2541 - val_accuracy: 0.8882
Epoch 4/80
144/144 [==============================] - 40s 279ms/step - loss: 0.2193 - accuracy: 0.9131 - val_loss: 0.1266 - val_accuracy: 0.9441
Epoch 5/80
144/144 [==============================] - 41s 288ms/step - loss: 0.1746 - accuracy: 0.9359 - val_loss: 0.1199 - val_accuracy: 0.9457
Epoch 6/80
144/144 [==============================] - 42s 288ms/step - loss: 0.1569 - accuracy: 0.9479 - val_loss: 0.2400 - val_accuracy: 0.9026
Epoch 7/80
144/144 [==============================] - 40s 280ms/step - loss: 0.1507 - accuracy: 0.9477 - val_loss: 0.0931 - val_accuracy: 0.9

KeyboardInterrupt: 

In [49]:
model.save("./learning_model.h5")

In [64]:
from tensorflow.keras.models import load_model
model = load_model('learning_model.h5')

In [65]:
import pandas as pd
import matplotlib.pyplot as plt

In [66]:
pd.DataFrame(history.history).plot(figsize= (8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()


TypeError: no numeric data to plot

In [67]:
y_pred=np.argmax(model.predict(X_test), axis=-1)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [68]:
y_test

array([1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,

In [69]:
accuracy= 0

for i in range(len(y_test)):
    if(y_test[i]== y_pred[i]).all():
        accuracy+= 1
        
print("accuracy: ", accuracy/len(y_test))

accuracy:  0.625


In [78]:
from sklearn.metrics import recall_score,f1_score
print(f1_score(y_test,y_pred))
print(recall_score(y_test,y_pred,pos_label=1))


1.0
0.7692307692307693
